In [1]:
import numpy as np
import pandas as pd
from utils.diff import diff

In [2]:
# 24. depr
# 半年频率。折旧除以固定资产。

# 53. pchdepr
# 半年频率。折旧变动百分比。

df1 = pd.read_csv('../../data/csmar/季_资产负债表.csv', usecols=['证券代码', '会计期间', '固定资产净额'], low_memory=False)
df2 = pd.read_csv('../../data/csmar/季_现金流量表(间接法).csv', usecols=['证券代码', '会计期间', '固定资产折旧、油气资产折耗、生产性生物资产折旧', '查询成功'], low_memory=False)
df2['固定资产折旧、油气资产折耗、生产性生物资产折旧'][df2['查询成功'].astype('bool')] = df2['固定资产折旧、油气资产折耗、生产性生物资产折旧'][df2['查询成功'].astype('bool')].fillna(0)
df3 = pd.merge(df1, df2, on=['证券代码', '会计期间'], how='left')
df3['半年'] = df3['会计期间'].apply(lambda x: True if x[5:7] in ['06', '12'] else False)
df3 = df3[df3['半年']].reset_index().drop(columns=['index'])
df3 = diff(df3, col=['固定资产折旧、油气资产折耗、生产性生物资产折旧'], freq='半年')
df3['depr'] = df3['固定资产折旧、油气资产折耗、生产性生物资产折旧']/df3['固定资产净额'].replace(0, np.nan)
df3['pchdepr'] = (df3['固定资产折旧、油气资产折耗、生产性生物资产折旧'] - df3.groupby('证券代码').shift(1)['固定资产折旧、油气资产折耗、生产性生物资产折旧']) / df3.groupby('证券代码').shift(1)['固定资产折旧、油气资产折耗、生产性生物资产折旧'].replace(0, np.nan)
df4 = df3[['证券代码', '会计期间', 'depr', 'pchdepr']]
df4.to_csv('../../output/csmar/半年_24_53.csv', index=False, encoding='utf-8-sig')
df4

,证券代码,会计期间,depr,pchdepr
0,1,1990-06-30,NaN,NaN
1,1,1990-12-31,NaN,NaN
2,1,1991-06-30,NaN,NaN
3,1,1991-12-31,NaN,NaN
4,1,1992-06-30,NaN,NaN
...,...,...,...,...
361675,900957,2020-12-31,0.040006,0.000095
361676,900957,2021-06-30,0.041761,0.002248
361677,900957,2021-12-31,0.043445,-0.000671
361678,900957,2022-06-30,0.045378,-0.000409
